In [1]:
pwd

'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification\\research'

In [2]:
import os
os.chdir("../")

In [3]:
pwd

'c:\\Users\\Dell\\Desktop\\Chicken\\Chicken-Disease-Classification'

In [4]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class TrainingConnfig:
    root_dir: Path
    trained_model_path: Path
    updated_base_model_path: Path
    training_data: Path
    params_epochs: int
    params_batch_size: int
    params_is_agumentation: bool
    params_image_size: list


@dataclass
class PrepareCallbacksConfig:
    root_dir : Path
    tensorboard_root_logs_dir : Path
    checkpoint_model_filepath : Path


In [5]:
from cnnClassifier.constants import *
from cnnClassifier.utils.common import read_yaml, create_directories
import tensorflow as tf

In [1]:
class ConfigurationManager:
    def __init__(
            self,
            config_file_path= CONFIG_FILE_PATH,
            params_file_path= PARAMS_FILE_PATH):
        
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)
        create_directories([self.config.artifacts_root])

    def get_prepare_callbacks_config(self) -> PrepareCallbacksConfig:
        config = self.config.prepare_callbacks
        model_ckpt_dir = os.path.dirname(config.checkpoint_model_filepath)
        create_directories([
            Path(model_ckpt_dir),
            Path(config.tensorboard_root_logs_dir)
        ])

        prepare_callbacks_config = PrepareCallbacksConfig(
            root_dir=Path(config.root_dir),
            tensorboard_root_logs_dir = Path(config.tensorboard_root_logs_dir),
            checkpoint_model_filepath = Path(config.checkpoint_model_filepath)
        )

        return prepare_callbacks_config
    
    def get_training_config(self)->TrainingConnfig:
        training = self.config.training
        prepare_base_model= self.config.prepare_base_model
        params = self.params
        training_data = os.path.join(self.config.data_ingestion.unzip_dir, "Chicken-fecal-images")
        create_directories([Path(training.root_dir)])

        training_config = TrainingConnfig(
            root_dir= Path(training.root_dir),
            trained_model_path= Path(training.trained_model_path),
            updated_base_model_path= Path(prepare_base_model.updated_base_model_path),
            training_data= Path(training_data),
            params_epochs= params.EPOCHS,
            params_batch_size= params.BATCH_SIZE,
            params_is_agumentation= params.AGUMENTATION,
            params_image_size= params.IMAGE_SIZE
        )

        return training_config

NameError: name 'CONFIG_FILE_PATH' is not defined

In [22]:
import time


In [23]:
class PrepareCallbacks:
    def __init__(self, config : PrepareCallbacksConfig):
        self.config = config

    @property
    def _create_tb_callbacks(self):
        timestamp = time.strftime("%Y-%m-%d-%H-%M-%S")
        tb_running_log_dir = os.path.join(
            self.config.tensorboard_root_logs_dir,
            f"tb_logs_at{timestamp}"
        )

        return tf.keras.callbacks.TensorBoard(log_dir=tb_running_log_dir)
    

    @property
    def _create_ckpt_callbacks(self):
        return tf.keras.callbacks.ModelCheckpoint(
            filepath=str(self.config.checkpoint_model_filepath),
            save_best_only=True
        )
    
    def get_tb_ckpt_callbacks(self):
        return[
            self._create_tb_callbacks,
            self._create_ckpt_callbacks
        ]


In [24]:
import os
from urllib import request
from zipfile import ZipFile
import tensorflow as tf
import time


In [40]:
from tensorflow.keras.models import Model
class Training:
    def __init__(self, config : TrainingConnfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.models.load_model(
            str(self.config.updated_base_model_path)
        )

        self.model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=0.001),
            loss=tf.keras.losses.BinaryCrossentropy(),
            metrics=['accuracy']
        )


   
    def train_valid_generator(self):
        datagenerator_kwargs = dict(
            rescale=1./255,
            validation_split=0.20
        )

        dataflow_kwargs = dict(
            target_size=self.config.params_image_size[:-1],
            batch_size=self.config.params_batch_size,
            interpolation= 'bilinear'
        )

        valid_datagenerator= tf.keras.preprocessing.image.ImageDataGenerator(
            **datagenerator_kwargs
        )

        self.valid_generator=valid_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=False,
            **dataflow_kwargs            
        )

        if self.config.params_is_agumentation:
            train_datagenerator=tf.keras.preprocessing.image.ImageDataGenerator(
                rotation_range=40,
                horizontal_flip=True,
                width_shift_range=0.2,
                height_shift_range=0.2,
                shear_range=0.2,
                zoom_range=0.2,
                **datagenerator_kwargs
            )

        else:
            train_datagenerator = valid_datagenerator

        self.train_generator = train_datagenerator.flow_from_directory(
            directory=self.config.training_data,
            subset='validation',
            shuffle=True,
            **dataflow_kwargs  
        )

    @staticmethod
    def save_model(path : Path, model : tf.keras.Model):
        model.save(str(path))
        

    def train(self, callback_list: list):
        self.steps_per_epochs =self.train_generator.samples //  self.train_generator.batch_size
        self.validation_steps = self.valid_generator.samples // self.valid_generator.batch_size

        self.model.fit(
            self.train_generator,
            steps_per_epoch=self.steps_per_epochs,
            epochs=self.config.params_epochs,
            validation_data=self.valid_generator,
            validation_steps=self.validation_steps,
            callbacks=callback_list
        )

        self.save_model(path=self.config.trained_model_path, model =self.model)

In [42]:
try:
    config = ConfigurationManager()
    prepare_callback_config = config.get_prepare_callbacks_config()
    prepare_callbacks = PrepareCallbacks(config=prepare_callback_config)
    callback_list = prepare_callbacks.get_tb_ckpt_callbacks()

    training_config = config.get_training_config()
    training = Training(config=training_config)
    training.get_base_model()
    training.train_valid_generator()
    training.train(
        callback_list=callback_list
    )
    
except Exception as e:
    raise e


[2023-07-17 19:01:57,702: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\config\config.yaml is successfully added:]
[2023-07-17 19:01:57,710: INFO: common: yaml file : c:\Users\Dell\Desktop\Chicken\Chicken-Disease-Classification\params.yaml is successfully added:]
['artifacts']
[2023-07-17 19:01:57,718: INFO: common: directory : artifacts is successfully created:]
[WindowsPath('artifacts/prepare_callbacks/checkpoint_dir'), WindowsPath('artifacts/prepare_callbacks/tensorboard_logs_dir')]
[2023-07-17 19:01:57,718: INFO: common: directory : artifacts\prepare_callbacks\checkpoint_dir is successfully created:]
[2023-07-17 19:01:57,718: INFO: common: directory : artifacts\prepare_callbacks\tensorboard_logs_dir is successfully created:]
[WindowsPath('artifacts/training')]
[2023-07-17 19:01:57,718: INFO: common: directory : artifacts\training is successfully created:]


[2023-07-17 19:01:58,599: WARNING: hdf5_format: No training configuration found in the save file, so the model was *not* compiled. Compile it manually.:]
Found 78 images belonging to 2 classes.
Found 78 images belonging to 2 classes.
4/4 [==============================] - 24s 7s/step - loss: 0.6805 - accuracy: 0.7258 - val_loss: 0.5326 - val_accuracy: 0.8125
